In [1]:
import numpy as np
import pickle
import operator
main_path = '/content/drive/My Drive/Colab Notebooks/'
eng = np.load('pad_eng.npy')
cns = np.load('pad_cns.npy')
cns_o = np.load('cns_o.npy', allow_pickle=True)
with open('vocab_bag.pkl', 'rb') as f:
    words = pickle.load(f)
with open('pad_word_to_index.pkl', 'rb') as f:
    word_to_index = pickle.load(f)
with open('pad_index_to_word.pkl', 'rb') as f:
    index_to_word = pickle.load(f)
vocab_size = len(word_to_index) + 1
maxLen=20
def get_file_list(file_path):
    dir_list = os.listdir(file_path)
    if not dir_list:
        return
    else:
        dir_list = sorted(dir_list, key=lambda x: os.path.getmtime(os.path.join(file_path, x)))
    return dir_list

In [2]:
from keras.preprocessing import sequence
def generate_train(batch_size):
    print('\n*********************************generate_train()*********************************')
    steps=0
    eng_ = eng
    cns_ = cns
    while True:
        batch_cns_o = cns_o[steps:steps+batch_size]
        batch_eng = eng_[steps:steps+batch_size]
        batch_cns = cns_[steps:steps+batch_size]
        outs = np.zeros([batch_size, maxLen, vocab_size], dtype='float32')
        for pos, i in enumerate(batch_cns_o):
            for pos_, j in enumerate(i):
                if pos_ > 20:
                    print(i)
                outs[pos, pos_, j] = 1 # one-hot
        yield [batch_eng, batch_cns], outs
        steps += batch_size
        if steps == 20000:
            steps = 0

Using TensorFlow backend.


In [3]:
from keras.layers import Embedding
from keras.layers import Input, Dense, LSTM, TimeDistributed, Bidirectional, Dropout, Concatenate, RepeatVector, Activation, Dot
from keras.layers import concatenate, dot                    
from keras.models import Model
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, TensorBoard,ReduceLROnPlateau
from keras.initializers import TruncatedNormal
import pydot
import os, re
truncatednormal = TruncatedNormal(mean=0.0, stddev=0.05)
embed_layer = Embedding(input_dim=vocab_size, 
                        output_dim=100, 
                        mask_zero=True,
                        input_length=None,
                        embeddings_initializer= truncatednormal)
LSTM_encoder = LSTM(256,
                      return_sequences=True,
                      return_state=True,
#                       activation='relu',
#                       dropout=0.25,
#                       recurrent_dropout=0.1,
                      kernel_initializer= 'lecun_uniform',
                      name='encoder_lstm'
                        )
LSTM_decoder = LSTM(256, 
                    return_sequences=True, 
                    return_state=True, 
#                     activation = 'relu',
#                     dropout=0.25, 
#                     recurrent_dropout=0.1,
                    kernel_initializer= 'lecun_uniform',
                    name='decoder_lstm'
                   )
#encoder输入 与 decoder输入
input_eng = Input(shape=(None, ), dtype='int32', name='input_eng')
input_cns = Input(shape=(None, ), dtype='int32', name='input_cns')

input_eng_embed = embed_layer(input_eng)
input_cns_embed = embed_layer(input_cns)

encoder_lstm, eng_h, eng_c = LSTM_encoder(input_eng_embed)

decoder_lstm, _, _ = LSTM_decoder(input_cns_embed, 
                                  initial_state=[eng_h, eng_c])

attention = dot([decoder_lstm, encoder_lstm], axes=[2, 2])
attention = Activation('softmax')(attention)
context = dot([attention, encoder_lstm], axes=[2,1])
decoder_combined_context = concatenate([context, decoder_lstm])

# output = dense1(decoder_combined_context)
# output = dense2(Dropout(0.5)(output))

# Has another weight + tanh layer as described in equation (5) of the paper
decoder_dense1 = TimeDistributed(Dense(128,activation="tanh"))
decoder_dense2 = TimeDistributed(Dense(vocab_size,activation="softmax"))
output = decoder_dense1(decoder_combined_context) # equation (5) of the paper
output = decoder_dense2(output) # equation (6) of the paper

model = Model([input_eng, input_cns], output)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.load_weights('models/W--189-0.5900-.h5')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_cns (InputLayer)          (None, None)         0                                            
__________________________________________________________________________________________________
input_eng (InputLayer)          (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    1775200     input_eng[0][0]                  
                                                                 input_cns[0][0]                  
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(Non

In [4]:
eng_model = Model(input_eng, [encoder_lstm, eng_h, eng_c])
eng_model.summary()
cns_h = Input(shape=(256,))
cns_c = Input(shape=(256,))
encoder_lstm = Input(shape=(maxLen,256))
target, h, c = LSTM_decoder(input_cns_embed, initial_state=[cns_h, cns_c])
attention = dot([target, encoder_lstm], axes=[2, 2])
attention_ = Activation('softmax')(attention)
context = dot([attention_, encoder_lstm], axes=[2,1])
decoder_combined_context = concatenate([context, target])
output = decoder_dense1(decoder_combined_context) # equation (5) of the paper
output = decoder_dense2(output) # equation (6) of the paper
cns_model = Model([input_cns, cns_h, cns_c, encoder_lstm], [output, h, c, attention_])
cns_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_eng (InputLayer)       (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 100)         1775200   
_________________________________________________________________
encoder_lstm (LSTM)          [(None, None, 256), (None 365568    
Total params: 2,140,768
Trainable params: 2,140,768
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_cns (InputLayer)          (None, None)         0                                            
__________________________________________________________________________________________________


In [5]:
from keras.preprocessing import sequence
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import jieba
import requests
def input_eng(seq):
    seq = seq.split(' ')
    sentence = seq
    seq = np.array([word_to_index[w] for w in seq])
    seq = sequence.pad_sequences([seq], maxlen=maxLen,
                                          padding='post', truncating='post')
    print(seq)
    return seq, sentence
def decode_greedy(seq, sentence):
    eng = seq
    cns = np.zeros((1, 1))
    attention_plot = np.zeros((20, 20))
    cns[0, 0] = word_to_index['BOS']
    i=1
    cns_ = []
    flag = 0
    encoder_lstm_, eng_h, eng_c = eng_model.predict(x=eng, verbose=1)
#     print(eng_h, '\n')
    while flag != 1:
        prediction, prediction_h, prediction_c, attention = cns_model.predict([
            cns, eng_h, eng_c, encoder_lstm_
        ])
        attention_weights = attention.reshape(-1, )
        attention_plot[i] = attention_weights
        word_arg = np.argmax(prediction[0, -1, :])#
        cns_.append(index_to_word[word_arg])
        if word_arg == word_to_index['EOS']  or i > 20:
            flag = 1
        cns = np.zeros((1, 1))
        cns[0, 0] = word_arg
        eng_h = prediction_h
        eng_c = prediction_c
        i += 1
    result = ' '.join(cns_)
#     attention_plot = attention_plot[:len(result.split(' ')), :len(sentence)]
#     plot_attention(attention_plot, sentence, result.split(' '))
    return ' '.join(cns_)

In [6]:
while True:
    seq = input()
    if seq == 'x':
        break
    seq, sentence = input_eng(seq)
    print(sentence)
    cns = decode_greedy(seq, sentence)
    print('翻译: ', cns)

Where are you ?
[[257 258  62  25   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]]
['Where', 'are', 'you', '?']
1/1 [==============================] - 0s 135ms/step
翻译:  你 在 哪儿 ? EOS
I love you !
[[  7 219  62   5   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]]
['I', 'love', 'you', '!']
1/1 [==============================] - 0s 4ms/step
翻译:  我 爱 你 。 EOS
Do you love me ?
[[114  62 219  30  25   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]]
['Do', 'you', 'love', 'me', '?']
1/1 [==============================] - 0s 3ms/step
翻译:  你 爱 我 吗 ？ EOS
x
